In [1]:
import GPyOpt
import numpy as np
import pandas as pd
import re

warning in stationary: failed to import cython module: falling back to numpy


In [2]:
KINIT_USERNAME = 'login'
KINIT_PASSWD = 'passwd'
N_EVENTS = 1000

# Optimization space

In [3]:
min_dist = 3.6

space = [{'name': 'pitch', 'type': 'continuous', 'domain': (min_dist, min_dist)},\
         {'name': 'yoffset_layer', 'type': 'continuous', 'domain': (min_dist/2, min_dist)},\
         {'name': 'yoffset_plane', 'type': 'continuous', 'domain': (min_dist*0.25, min_dist*1.25)},\
         {'name': 'zshift_layer', 'type': 'continuous', 'domain': (1.6, 2.6)},\
         {'name': 'zshift_plane', 'type': 'continuous', 'domain': (3.8, 6.8)},\
         {'name': 'zshift_view', 'type': 'continuous', 'domain': (10, 10)},\
         {'name': 'alpha', 'type': 'discrete', 'domain': (5, 5)}]

constraints = [{'name': 'constr_1', 'constrain': '-(x[:,0]-x[:,1])**2-x[:,3]**2+2**2'},\
               {'name': 'constr_2', 'constrain': '-(x[:,1]-x[:,2])**2-(x[:,3]-x[:,4])**2+2**2'},\
               {'name': 'constr_3', 'constrain': 'x[:,3]+x[:,4]+2-x[:,5]'}]

In [4]:
feasible_region = GPyOpt.Design_space(space=space, constraints=constraints)

np.random.seed(42)

# Import skygrid client

In [5]:
import time
import json

from disneylandClient import (
    new_client,
    Job,
    RequestWithId,
)

In [6]:
STATUS_IN_PROCESS = set([
    Job.PENDING,
    Job.PULLED,
    Job.RUNNING,
])
STATUS_FINAL = set([
    Job.COMPLETED,
    Job.FAILED,
])

In [7]:
def return_descriptor(point):
    
    pitch, yoffset_layer, yoffset_plane, zshift_layer, zshift_plane, zshift_view, alpha = point
    
    logining = "sh -lc 'echo "+KINIT_PASSWD+" | kinit "+KINIT_USERNAME+"; "
    sourcing = "source /opt/FairShipRun/config.sh; "
    simulation = "python $SHIPOPT/code/objective.py --pitch "+str(pitch)+" --yoffset_layer "+str(yoffset_layer)+\
              " --yoffset_plane "+str(yoffset_plane)+" --zshift_layer "+str(zshift_layer)+" --zshift_plane "+\
              str(zshift_plane)+" --zshift_view "+str(zshift_view)+" --alpha "+str(int(alpha))+\
              " --nEvents "+str(N_EVENTS)+" --output /output/output.txt'"
    cmd = logining+sourcing+simulation

    descriptor = {
        "descriptor": {
            "input": [],

            "container": {
                "workdir": "",
                "name": "oleg94/worker",
                "cpu_needed": 1,
                "max_memoryMB": 4096,
                "min_memoryMB": 1024,
                "cmd": cmd,
            },

            "required_outputs": {
                "output_uri": "none:",
                "file_contents": [
                    {"file": "output.txt", "to_variable": "out"}
                ]
            }
        }
    }
    
    return descriptor

# Initial design

In [9]:
n_estimators = 80
n_initial_design = 200

In [ ]:
initial_design = GPyOpt.experiment_design.initial_design('random', feasible_region, n_initial_design)

In [10]:
initial_objective = np.zeros(n_initial_design)

In [11]:
stub = new_client()
jobs = []

In [ ]:
init_d_i = 0

for epoch in range(n_initial_design // n_estimators):
    
    print("EPOCH #"+str(epoch)+" started.")
    
    epoch_jobs = [0] * n_estimators
    
    for k in range(n_estimators):
        descriptor = return_descriptor(initial_design[init_d_i])
        init_d_i += 1
        epoch_jobs[k] = Job(input=json.dumps(descriptor), kind="docker")
        epoch_jobs[k] = stub.CreateJob(epoch_jobs[k])
    
    prev_number_of_finished_jobs = 0
    prev_number_of_running_jobs = 0
    prev_number_of_pending_jobs = 0
    
    while True:
        for k in range(n_estimators):
            epoch_jobs[k] = stub.GetJob(RequestWithId(id=epoch_jobs[k].id))
        
        number_of_finished_jobs = 0
        number_of_running_jobs = 0
        number_of_pending_jobs = 0
        for k in range(n_estimators):
            if epoch_jobs[k].status in STATUS_FINAL:
                number_of_finished_jobs += 1
            if epoch_jobs[k].status == Job.PENDING:
                number_of_pending_jobs += 1
            if epoch_jobs[k].status == Job.RUNNING:
                number_of_running_jobs += 1
                
        if (number_of_finished_jobs != prev_number_of_finished_jobs) or (prev_number_of_running_jobs != number_of_running_jobs) or (prev_number_of_pending_jobs != number_of_pending_jobs):
            print("Finished jobs: "+str(number_of_finished_jobs)+\
                  " Running jobs: "+str(number_of_running_jobs)+\
                  " Pending jobs: "+str(number_of_pending_jobs))
            prev_number_of_finished_jobs = number_of_finished_jobs
            prev_number_of_running_jobs = number_of_running_jobs
            prev_number_of_pending_jobs = number_of_pending_jobs
            
        if number_of_finished_jobs == n_estimators:
            break
        time.sleep(120)
    
    jobs += epoch_jobs

EPOCH #0 started.
Finished jobs: 0 Running jobs: 0 Pending jobs: 20
Finished jobs: 0 Running jobs: 20 Pending jobs: 0


In [45]:
df_init_design = pd.DataFrame(initial_design, columns=['pitch', 'yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane', 'zshift_view', 'alpha'])
df_init_design['objective'] = [float(re.sub('[\[\]"variable:out=]', '', job.output)) if re.sub('[\[\]"variable:out=]', '', job.output) else np.nan for job in jobs]

In [47]:
df_init_design.to_csv('observations/observations.csv', index=False)

# Main part of optimization

In [14]:
df_init_design = pd.read_csv('observations/observations.csv')
n_estimators = 10
n_epochs = 100
stub = new_client()

In [15]:
for epoch in range(n_epochs):
    
        print("EPOCH #"+str(epoch)+" started.")
    
        epoch_jobs = [0] * n_estimators
        
        pending_X = []

        for k in range(n_estimators):
            
            step_X = df_init_design[df_init_design.columns[:-1]].values
            #because we want to maximize
            step_Y = -df_init_design[df_init_design.columns[-1:]].values
            ignored_X = step_X[np.isnan(step_Y.ravel())]
            step_X = step_X[~np.isnan(step_Y.ravel())]
            step_Y = step_Y[~np.isnan(step_Y.ravel())]
            bo = GPyOpt.methods.BayesianOptimization(f=None, domain=space, constraints=constraints, X=step_X,\
                                                     Y=step_Y, de_duplication=True, initial_design_numdata=200)
            new_point = bo.suggest_next_locations(pending_X=np.array(pending_X), ignored_X=ignored_X)[0]

            descriptor = return_descriptor(new_point)
            epoch_jobs[k] = Job(input=json.dumps(descriptor), kind="docker")
            epoch_jobs[k] = stub.CreateJob(epoch_jobs[k])
            
            pending_X += [new_point]
        
        prev_number_of_finished_jobs = 0
        prev_number_of_running_jobs = 0
        prev_number_of_pending_jobs = 0

        while True:
            for k in range(n_estimators):
                epoch_jobs[k] = stub.GetJob(RequestWithId(id=epoch_jobs[k].id))

            number_of_finished_jobs = 0
            number_of_running_jobs = 0
            number_of_pending_jobs = 0
            for k in range(n_estimators):
                if epoch_jobs[k].status in STATUS_FINAL:
                    number_of_finished_jobs += 1
                if epoch_jobs[k].status == Job.PENDING:
                    number_of_pending_jobs += 1
                if epoch_jobs[k].status == Job.RUNNING:
                    number_of_running_jobs += 1

            if (number_of_finished_jobs != prev_number_of_finished_jobs) or (prev_number_of_running_jobs != number_of_running_jobs) or (prev_number_of_pending_jobs != number_of_pending_jobs):
                print("Finished jobs: "+str(number_of_finished_jobs)+\
                      " Running jobs: "+str(number_of_running_jobs)+\
                      " Pending jobs: "+str(number_of_pending_jobs))
                prev_number_of_finished_jobs = number_of_finished_jobs
                prev_number_of_running_jobs = number_of_running_jobs
                prev_number_of_pending_jobs = number_of_pending_jobs

            if number_of_finished_jobs == n_estimators:
                break
            time.sleep(120)
            
        for k, point in enumerate(pending_X):
        
            value = float(re.sub('[\[\]"variable:out=]', '', epoch_jobs[k].output)) if re.sub('[\[\]"variable:out=]', '', epoch_jobs[k].output) else np.nan
        
            df_init_design.loc[len(df_init_design)] = list(point)+[value]
            df_init_design.to_csv('observations/observations.csv', index=False)

EPOCH #0 started.
Finished jobs: 0 Running jobs: 0 Pending jobs: 1
Finished jobs: 1 Running jobs: 0 Pending jobs: 0
